In [ ]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

/home/ubuntu/varios/skforecast


In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.preprocessing import RollingFeatures

In [4]:
series = pd.DataFrame({'l1': pd.Series(np.arange(10)), 
                       'l2': pd.Series(np.arange(10))})

In [77]:
encoding = None

rng = np.random.default_rng(12345)
series = pd.DataFrame({"l1": rng.normal(10, 3, 20), "l2": rng.normal(10, 3, 20)})
forecaster = ForecasterRecursiveMultiSeries(
    LinearRegression(), lags=3, encoding=encoding, binner_kwargs={"n_bins": 3}
)
forecaster.fit(series=series)
y_true = {"l1": rng.normal(10, 3, 20), "l2": rng.normal(10, 3, 20)}
y_pred = {"l1": rng.normal(10, 3, 20), "l2": rng.normal(10, 3, 20)}

forecaster.set_out_sample_residuals(y_true=y_true, y_pred=y_pred)
expected_out_sample = {
    'l1': np.array(
        [1.60962553,  1.71263736,  9.19708889, -2.73386424,  1.51464604,
        -3.32699998,  3.5987987 ,  0.13608979, -2.06018676, -2.72988393,
         1.63221968, -0.57130461, -1.08106429, -1.45886519,  0.91789988,
        -2.19933831,  1.97853178, -2.56217282,  3.69683321, -4.86497532]),
    'l2': np.array(
            [0.51984518,  7.40971902, -0.77881843, -2.89834312, -4.09573881,
            2.98823132,  2.43787115, -5.81851232,  1.34536138, -2.88085653,
            -8.26701406, -8.9861346 ,  0.31671219, -0.83725661, -4.37490671,
            -2.88874692,  0.50604084, -3.41273397, -2.71056093,  3.120254 ]),
    '_unknown_level':  np.array(
            [1.60962553,  1.71263736,  9.19708889, -2.73386424,  1.51464604,
            -3.32699998,  3.5987987 ,  0.13608979, -2.06018676, -2.72988393,
            1.63221968, -0.57130461, -1.08106429, -1.45886519,  0.91789988,
            -2.19933831,  1.97853178, -2.56217282,  3.69683321, -4.86497532,
            0.51984518,  7.40971902, -0.77881843, -2.89834312, -4.09573881,
            2.98823132,  2.43787115, -5.81851232,  1.34536138, -2.88085653,
            -8.26701406, -8.9861346 ,  0.31671219, -0.83725661, -4.37490671,
            -2.88874692,  0.50604084, -3.41273397, -2.71056093,  3.120254])
        }

expected_out_sample_by_bin = {
    "l1": {
        0: np.array([
            1.60962553, 1.71263736, 9.19708889, 1.51464604, 3.5987987,
            1.63221968, -2.19933831, 1.97853178, -2.56217282, 3.69683321,
        ]),
        1: np.array([
            -3.32699998,
        ]),
        2: np.array([
            -2.73386424, 0.13608979, -2.06018676, -2.72988393, -0.57130461,
            -1.08106429, -1.45886519, 0.91789988, -4.86497532,
        ]),
    },
    "l2": {
        0: np.array([
            0.51984518, 7.40971902, -4.09573881, 2.98823132, -2.88085653,
            3.120254,
        ]),
        1: np.array([
            -0.77881843, 2.43787115, 0.31671219, -2.71056093,
        ]),
        2: np.array([
            -2.89834312, -5.81851232, 1.34536138, -8.26701406, -8.9861346,
            -0.83725661, -4.37490671, -2.88874692, 0.50604084, -3.41273397,
        ]),
    },
    "_unknown_level": {
        0: np.array([
            1.60962553, 1.71263736, 9.19708889, 1.51464604, 3.5987987,
            1.63221968, -2.19933831, 1.97853178, -2.56217282, 3.69683321,
            0.51984518, 7.40971902, -4.09573881, 2.98823132, -2.88085653,
            3.120254,
        ]),
        1: np.array([
            -3.32699998, -0.77881843, 2.43787115, 0.31671219, -2.71056093,
        ]),
        2: np.array([
            -2.73386424, 0.13608979, -2.06018676, -2.72988393, -0.57130461,
            -1.08106429, -1.45886519, 0.91789988, -4.86497532, -2.89834312,
            -5.81851232, 1.34536138, -8.26701406, -8.9861346, -0.83725661,
            -4.37490671, -2.88874692, 0.50604084, -3.41273397,
        ]),
    },
}

if encoding is None:
    expected_out_sample = {'_unknown_level': expected_out_sample['_unknown_level']}
    expected_out_sample_by_bin = {
        '_unknown_level': {
            0: np.array([ 1.60962553,  1.71263736,  9.19708889,  1.51464604,  3.5987987 ,
                    1.63221968, -2.19933831,  1.97853178, -2.56217282,  3.69683321,
                    0.51984518,  7.40971902, -4.09573881,  2.98823132, -2.88085653,
                    3.120254  ]),
            1: np.array([-3.32699998, -0.77881843,  2.43787115,  0.31671219, -2.71056093]),
            2: np.array([-2.73386424,  0.13608979, -2.06018676, -2.72988393, -0.57130461,
                    -1.08106429, -1.45886519,  0.91789988, -4.86497532, -2.89834312,
                    -5.81851232,  1.34536138, -8.26701406, -8.9861346 , -0.83725661,
                    -4.37490671, -2.88874692,  0.50604084, -3.41273397])
        }
    }
    
assert expected_out_sample.keys() == forecaster.out_sample_residuals_.keys()
assert all(
    np.allclose(expected_out_sample[k], forecaster.out_sample_residuals_[k])
    for k in expected_out_sample.keys()
)

for k in expected_out_sample_by_bin.keys():
    for bin in expected_out_sample_by_bin[k].keys():
        assert np.allclose(
            expected_out_sample_by_bin[k][bin],
            forecaster.out_sample_residuals_by_bin_[k][bin],
        )

/home/ubuntu/varios/skforecast/skforecast/recursive/_forecaster_recursive_multiseries.py:468: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(
/home/ubuntu/varios/skforecast/skforecast/recursive/_forecaster_recursive_multiseries.py:3233: UnknownLevelWarning: As `encoding` is set to `None`, no distinction between levels is made. All residuals are stored in the '_unknown_level' key. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=UnknownLevelWarning)
  warnings.warn(


AssertionError: 

In [72]:
k

'_unknown_level'

In [74]:
forecaster.out_sample_residuals_by_bin_[k]

{0: array([ 1.60962553,  1.71263736,  9.19708889,  1.51464604,  3.5987987 ,
         1.97853178, -2.56217282,  3.69683321,  0.51984518,  7.40971902,
        -0.77881843, -4.09573881,  2.98823132,  2.43787115, -2.88085653,
         3.120254  ]),
 1: array([-3.32699998,  1.63221968, -2.19933831,  0.31671219, -2.71056093]),
 2: array([-2.73386424,  0.13608979, -2.06018676, -2.72988393, -0.57130461,
        -1.08106429, -1.45886519,  0.91789988, -4.86497532, -2.89834312,
        -5.81851232,  1.34536138, -8.26701406, -8.9861346 , -0.83725661,
        -4.37490671, -2.88874692,  0.50604084, -3.41273397])}

In [76]:
expected_out_sample_by_bin

{'_unknown_level': {0: array([ 1.60962553,  1.71263736,  9.19708889,  1.51464604,  3.5987987 ,
          1.63221968, -2.19933831,  1.97853178, -2.56217282,  3.69683321,
          0.51984518,  7.40971902, -4.09573881,  2.98823132, -2.88085653,
          3.120254  ]),
  1: array([-3.32699998, -0.77881843,  2.43787115,  0.31671219, -2.71056093]),
  2: array([-2.73386424,  0.13608979, -2.06018676, -2.72988393, -0.57130461,
         -1.08106429, -1.45886519,  0.91789988, -4.86497532, -2.89834312,
         -5.81851232,  1.34536138, -8.26701406, -8.9861346 , -0.83725661,
         -4.37490671, -2.88874692,  0.50604084, -3.41273397])}}